In [1]:
!pip install optgbm

In [2]:
import time
import datetime

import numpy as np
import pandas as pd
import optgbm as opt

from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

non_id_columns = train_df.columns[train_df.columns != 'id']

In [4]:
from sklearn.preprocessing import LabelEncoder
features_target = ['target'] 

for feature in features_target:
    le = LabelEncoder()
    le.fit(train_df[feature])
    train_df[feature] = le.transform(train_df[feature])
    
print(train_df['target'].head())

0    5
1    5
2    1
3    7
4    1
Name: target, dtype: int64


In [5]:
train_knn = np.load("add_feat_train.npy")
test_knn = np.load("add_feat_test.npy")

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_knn = scaler.fit_transform(train_knn)
test_knn = scaler.transform(test_knn)

In [6]:
train_df = pd.concat([train_df, 
                      pd.DataFrame(train_knn, columns = ['knn_1', 'knn_2', 'knn_3', 'knn_4', 'knn_5', 'knn_6', 'knn_7', 'knn_8', 'knn_9'])], axis = 1)
test_df = pd.concat([test_df, 
                     pd.DataFrame(test_knn, columns = ['knn_1', 'knn_2', 'knn_3', 'knn_4', 'knn_5', 'knn_6', 'knn_7', 'knn_8', 'knn_9'])], axis = 1,)

In [7]:
print(len(train_df))
train_df = train_df.drop_duplicates(subset=non_id_columns)
print(len(train_df))

200000
199894


In [8]:
def do_run(i, train_df, test_df):
    print('starting run', i)
    print(datetime.datetime.now())
    start = time.time()

    X_train = train_df.drop(['id', 'target'], 1)
    y_train = train_df['target']

    opt_obgm = opt.OGBMClassifier()
    opt_obgm.fit(X_train, y_train)

    minutes = (time.time() - start) / 60
    print(round(minutes, 2))
    print(datetime.datetime.now())
    
    X_test = test_df.drop('id', 1)
    test_preds = opt_obgm.predict_proba(X_test)
    return test_preds

In [9]:
n_runs = 5
test_result = do_run(1, train_df, test_df)

for i in range(2, n_runs+1):
    loop_test_result = do_run(i, train_df, test_df)
    test_result = test_result + loop_test_result

test_result = test_result / n_runs

/opt/conda/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classifier=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
[I 2021-06-30 02:54:32,739] A new study created in memory with name: no-name-cee4b835-80bc-4fc4-ae45-f2e04e386798
Searching the best hyperparameters...


starting run 1
2021-06-30 02:54:32.695141


[I 2021-06-30 02:55:38,338] Trial 0 finished with value: 1.7513021276673264 and parameters: {'feature_fraction': 0.4, 'max_depth': 3, 'num_leaves': 8, 'min_data_in_leaf': 12904, 'lambda_l1': 0.07020973712676813, 'lambda_l2': 0.012657687970392292, 'bagging_fraction': 0.55, 'bagging_freq': 3}. Best is trial 0 with value: 1.7513021276673264.
[I 2021-06-30 02:56:40,811] Trial 1 finished with value: 1.7518556227177657 and parameters: {'feature_fraction': 0.8, 'max_depth': 7, 'num_leaves': 122, 'min_data_in_leaf': 803, 'lambda_l1': 7.032334200360548e-07, 'lambda_l2': 0.0008873230778094086, 'bagging_fraction': 0.8, 'bagging_freq': 9}. Best is trial 0 with value: 1.7513021276673264.
[I 2021-06-30 02:58:18,159] Trial 2 finished with value: 1.7564587851661713 and parameters: {'feature_fraction': 0.35, 'max_depth': 2, 'num_leaves': 3, 'min_data_in_leaf': 29085, 'lambda_l1': 0.008314788968686737, 'lambda_l2': 6.387930101696766e-07, 'bagging_fraction': 0.9, 'bagging_freq': 4}. Best is trial 0 with 

[I 2021-06-30 03:23:14,470] Trial 23 finished with value: 1.748074106455297 and parameters: {'feature_fraction': 0.30000000000000004, 'max_depth': 5, 'num_leaves': 19, 'min_data_in_leaf': 6502, 'lambda_l1': 0.007083089616871473, 'lambda_l2': 4.306918746061075e-09, 'bagging_fraction': 0.7, 'bagging_freq': 5}. Best is trial 11 with value: 1.747840315589913.
[I 2021-06-30 03:24:25,668] Trial 24 finished with value: 1.7490699849303168 and parameters: {'feature_fraction': 0.15000000000000002, 'max_depth': 7, 'num_leaves': 78, 'min_data_in_leaf': 1205, 'lambda_l1': 3.601572665461864e-06, 'lambda_l2': 4.4075044229816723e-07, 'bagging_fraction': 0.6, 'bagging_freq': 6}. Best is trial 11 with value: 1.747840315589913.
[I 2021-06-30 03:26:07,225] Trial 25 finished with value: 1.748233307382487 and parameters: {'feature_fraction': 0.35, 'max_depth': 6, 'num_leaves': 44, 'min_data_in_leaf': 3451, 'lambda_l1': 0.0001676762975841428, 'lambda_l2': 9.35372711495996e-09, 'bagging_fraction': 0.75, 'bagg

52.4
2021-06-30 03:46:56.919648


...

/opt/conda/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classifier=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
[I 2021-06-30 06:34:07,678] A new study created in memory with name: no-name-9ba7c46e-3132-4b92-b03b-d4861c4a0078
Searching the best hyperparameters...


starting run 5
2021-06-30 06:34:07.609394


[I 2021-06-30 06:35:18,334] Trial 0 finished with value: 1.7491329643644096 and parameters: {'feature_fraction': 0.9, 'max_depth': 7, 'num_leaves': 39, 'min_data_in_leaf': 5107, 'lambda_l1': 0.14780729071539553, 'lambda_l2': 2.9307285864920956, 'bagging_fraction': 0.7, 'bagging_freq': 7}. Best is trial 0 with value: 1.7491329643644096.
[I 2021-06-30 06:36:34,840] Trial 1 finished with value: 1.7567538747447144 and parameters: {'feature_fraction': 0.25, 'max_depth': 3, 'num_leaves': 5, 'min_data_in_leaf': 28187, 'lambda_l1': 1.6525974317150166e-08, 'lambda_l2': 5.453302061476086e-05, 'bagging_fraction': 0.8, 'bagging_freq': 6}. Best is trial 0 with value: 1.7491329643644096.
[I 2021-06-30 06:37:05,894] Trial 2 finished with value: 1.7628293376093196 and parameters: {'feature_fraction': 0.9, 'max_depth': 3, 'num_leaves': 2, 'min_data_in_leaf': 23948, 'lambda_l1': 2.4424278966580415, 'lambda_l2': 9.411997737278678e-06, 'bagging_fraction': 0.8, 'bagging_freq': 5}. Best is trial 0 with valu

[I 2021-06-30 07:01:31,410] Trial 23 finished with value: 1.7487649981434696 and parameters: {'feature_fraction': 0.8, 'max_depth': 5, 'num_leaves': 21, 'min_data_in_leaf': 9316, 'lambda_l1': 9.854054867555123e-08, 'lambda_l2': 0.0014218460849587844, 'bagging_fraction': 0.9, 'bagging_freq': 9}. Best is trial 11 with value: 1.748572892194921.
[I 2021-06-30 07:02:51,865] Trial 24 finished with value: 1.7486618880761235 and parameters: {'feature_fraction': 0.85, 'max_depth': 6, 'num_leaves': 34, 'min_data_in_leaf': 5867, 'lambda_l1': 3.8013671481870947e-06, 'lambda_l2': 0.023308185144919363, 'bagging_fraction': 0.95, 'bagging_freq': 8}. Best is trial 11 with value: 1.748572892194921.
[I 2021-06-30 07:04:08,606] Trial 25 finished with value: 1.7490759185915863 and parameters: {'feature_fraction': 1.0, 'max_depth': 6, 'num_leaves': 34, 'min_data_in_leaf': 5765, 'lambda_l1': 4.621686937753445e-06, 'lambda_l2': 0.02423231884550369, 'bagging_fraction': 0.95, 'bagging_freq': 8}. Best is trial 1

48.94
2021-06-30 07:23:04.062426


In [11]:
submission = pd.DataFrame(test_result)
submission.columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
submission['id'] = test_df['id']

submission.to_csv("submission_opt_obgm_knn_ensemble.csv", index=False)